In [1]:
# scikit learn 방식

In [2]:
import mlflow
mlflow.__version__

'3.1.0'

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [9]:
iris = load_iris()
x_train, x_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size = 0.2, random_state=42
)

with mlflow.start_run():
    # 모델 학습
    clf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
    clf.fit(x_train, y_train)


    # 예측및평가
    y_pred = clf.predict(x_test)
    acc = accuracy_score(y_test, y_pred)

    # 메트릭, 모델 로깅
    mlflow.log_param('n_estimators', 100)
    mlflow.log_param('max_depth', 3)
    mlflow.log_metric('accuracy', acc)

    mlflow.sklearn.log_model(clf, 'model')
    print('logged model accuracy: ', acc)

2025/06/11 15:34:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/11 15:34:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/06/11 15:34:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


logged model accuracy:  1.0
🏃 View run crawling-flea-596 at: http://127.0.0.1:5000/#/experiments/0/runs/e956229a679149d0a7ff62ea84767198
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


In [10]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient

# MLflow 설정
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment("Iris-Sklearn-RandomSearch")

# 데이터 로드
iris = load_iris(as_frame=True)
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 파이프라인 정의
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(random_state=42))
])

# 하이퍼파라미터 탐색
param_grid = {
    'clf__n_estimators': [50, 100, 150],
    'clf__max_depth': [2, 3, 4, 5, 6]
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=5,
    cv=3,
    random_state=42
)
search.fit(X_train, y_train)

# 예시 입력/서명 추출
input_example = X_test[:5]
y_pred = search.predict(X_test)
signature = infer_signature(X_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

# MLflow Run
with mlflow.start_run(run_name="random_forest_search") as run:
    mlflow.log_params(search.best_params_)
    mlflow.log_metric("accuracy", accuracy)

    # 모델 저장
    model_info = mlflow.sklearn.log_model(
        sk_model=search.best_estimator_,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # 모델 등록
    registered_model_name = "iris_rf_model"
    result = mlflow.register_model(
        model_uri=model_info.model_uri,
        name=registered_model_name
    )

    version = result.version
    print(f"모델 등록 완료: {registered_model_name}, version {version}")

# ---------------------------------------------------
# 모델 설명, 태그, 별칭 추가 (등록 이후 별도로 수행)
# ---------------------------------------------------
client = MlflowClient()

# 등록 모델 설명 업데이트
client.update_registered_model(
    name=registered_model_name,
    description="Random Forest 모델로 Iris 품종을 분류합니다. 전처리 포함된 sklearn 파이프라인 사용."
)

# 버전별 태그 추가
client.set_model_version_tag(registered_model_name, version, "accuracy", f"{accuracy:.4f}")
client.set_model_version_tag(registered_model_name, version, "cv", "3-fold")
client.set_model_version_tag(registered_model_name, version, "model_type", "RandomForest")

# 별칭 설정 (가장 최신 버전을 Production으로 지정)
client.set_registered_model_alias(
    name=registered_model_name,
    alias="production",
    version=version
)

print(f"설명, 태그, alias 설정 완료: version {version}")

2025/06/11 15:41:34 INFO mlflow.tracking.fluent: Experiment with name 'Iris-Sklearn-RandomSearch' does not exist. Creating a new experiment.
2025/06/11 15:41:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/11 15:41:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
C:\Users\Admin\Desktop\LGSW\LGSW02\ml_dl\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Successfully registered model 'iris_rf_model'.
2025/06/11 15:41:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_rf_model, version 1


모델 등록 완료: iris_rf_model, version 1
🏃 View run random_forest_search at: http://127.0.0.1:5000/#/experiments/370467540616997944/runs/e4bca73704ab4a4ba24419f98e28fd00
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/370467540616997944
설명, 태그, alias 설정 완료: version 1


Created version '1' of model 'iris_rf_model'.
